In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib
import google.generativeai as genai
import os

/home/s1m/Documents/SIT/S2/FASI/TP_FASI/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset
df = pd.read_excel("corrections2.xlsx", sheet_name="Réponses au formulaire 1")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 44 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Horodateur                                            39 non-null     datetime64[ns]
 1   nom prénom                                            39 non-null     object        
 2   numero du dossier                                     39 non-null     object        
 3   date de l'intervention                                39 non-null     datetime64[ns]
 4   Age lors de l'intervention                            39 non-null     int64         
 5   sexe                                                  39 non-null     int64         
 6   IMC (Kg/m2)                                           37 non-null     object        
 7   SC (m2)                                               38 non-null     float64     

In [4]:
df.tail()

,Horodateur,nom prénom,numero du dossier,date de l'intervention,Age lors de l'intervention,sexe,IMC (Kg/m2),SC (m2),OBESITE,Comorbidités,...,Durée de séjour total,Décès en post-opératoire,masse VG g/m2,Surface valvulaire aortique indexée : cm²/m2,ANATOMIE DE LA VALVE ECHO,ANATOMIE DE LA VALVE CHIRURGIE,STRAIN VG GLS : %,ATCD CHIRURGICAUX CARDIAQUES,CHIRURGIE CARDIAQUE PRECEDENTE:,DIAMETRE SOUS AO OU ANNEAU AO
34,2025-05-01 18:18:52,MECENE Ayoub,138/2024,2024-05-27,19,1,16,1.70,0.0,0,...,30,0,163.00,0.39,BICUSPIDE,BICUSPIDE,NaN,NaN,NaN,NaN
35,2025-05-02 11:47:12,CHIKHAOUI MESSOUDA,061/2023,2023-03-26,46,0,34.9,2.02,1.0,0,...,3,1,84.00,0.30,TRICUSPIDE,TRICUSPIDE,NaN,NaN,NaN,NaN
36,2025-05-02 12:36:12,BELAHDJEL CHAMSEDDINE,260/2023,2023-12-27,27,1,20,1.66,0.0,0,...,23,0,169.00,0.50,BICUSPIDE,BICUSPIDE,-3.2,REDUX,STENOSE SUPRAVALVULAIRE AO OPEREE 2009 ELARGIS...,24.0
37,2025-05-02 22:48:37,BENZAOUI TAYEB,094/2023,2023-06-15,65,1,27.7,1.92,0.0,0,...,43,0,147.39,0.30,NaN,NaN,NaN,NaN,NaN,21.0
38,2025-05-02 23:00:31,BOUDEJLJEL Ameur,114/2023,2023-06-11,44,1,2025-01-29 00:00:00,2.06,0.0,0,...,26,0,133.06,0.50,NaN,NaN,NaN,1ERE CHIRURGIE,NaN,22.0


In [5]:
import numpy as np

def add_gaussian_noise(df, columns, noise_level=0.05):
    df_aug = df.copy()
    for col in columns:
        if df[col].dtype in [np.float64, np.int64]:
            noise = np.random.normal(0, noise_level * df[col].std(), len(df))
            df_aug[col] = df[col] + noise
    return df_aug

# Columns to augment
numeric_cols = ['Age lors de l\'intervention', 'IMC (Kg/m2)', 'SC (m2)', 
               'Fraction d\'éjection VG : %', 'masse VG g/m2']
aug_df = add_gaussian_noise(df, numeric_cols)

KeyError: "Fraction d'éjection VG : %"

In [6]:
from imblearn.over_sampling import SMOTE

def apply_smote(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    return pd.concat([X_res, y_res], axis=1)

# Example for mortality prediction
if 'Décès en post-opératoire' in df.columns:
    aug_df = apply_smote(df, 'Décès en post-opératoire')

TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [22]:
from faker import Faker

def augment_categorical(df, cat_columns):
    fake = Faker()
    df_aug = df.copy()
    for col in cat_columns:
        unique_vals = df[col].unique()
        df_aug[col] = df[col].apply(lambda x: x if np.random.random() > 0.3 
                                   else np.random.choice(unique_vals))
    return df_aug

cat_cols = ['Type de chirurgie :', 'Type de prothèse implantée', 
            'ANATOMIE DE LA VALVE ECHO']
aug_df = augment_categorical(df, cat_cols)

In [29]:
aug_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 44 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Horodateur                                            39 non-null     datetime64[ns]
 1   nom prénom                                            39 non-null     object        
 2   numero du dossier                                     39 non-null     object        
 3   date de l'intervention                                39 non-null     datetime64[ns]
 4   Age lors de l'intervention                            39 non-null     int64         
 5   sexe                                                  39 non-null     int64         
 6   IMC (Kg/m2)                                           37 non-null     object        
 7   SC (m2)                                               38 non-null     float64     

In [26]:
def perturb_dates(df, date_columns, days_range=7):
    df_aug = df.copy()
    for col in date_columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            days_noise = np.random.randint(-days_range, days_range, len(df))
            df_aug[col] = df[col] + pd.to_timedelta(days_noise, unit='D')
    return df_aug

date_cols = ['date de l\'intervention']
aug_df = perturb_dates(df, date_cols)

In [17]:
medication_db = ['Aspirin', 'Metoprolol', 'Atorvastatin', 'Warfarin', 
                'Furosemide', 'Ramipril', 'Insulin']

def augment_medications(df, med_column):
    df_aug = df.copy()
    for i, row in df.iterrows():
        if pd.notna(row[med_column]):
            meds = str(row[med_column]).split(', ')
            if len(meds) > 0 and np.random.random() > 0.7:
                new_med = np.random.choice(medication_db)
                if new_med not in meds:
                    meds.append(new_med)
                df_aug.at[i, med_column] = ', '.join(meds)
    return df_aug

aug_df = augment_medications(df, 'Traitements médicamenteux  pré-opératoires :')

In [32]:
def medical_augmentation_pipeline(df, augmentation_factor=2):
    # Get exact column names from the DataFrame
    numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
    date_cols = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
    cat_cols = ['Type de chirurgie :', 'Type de prothèse implantée', 
                'ANATOMIE DE LA VALVE ECHO']  # Adjust as needed
    
    augmented_dfs = []
    
    for _ in range(augmentation_factor):
        # Numerical augmentation
        temp_df = add_gaussian_noise(df, numeric_cols, noise_level=0.03)
        
        # Categorical augmentation
        temp_df = augment_categorical(temp_df, cat_cols)
        
        # Date perturbation
        temp_df = perturb_dates(temp_df, date_cols, days_range=5)
        
        # Medication augmentation if column exists
        med_col = 'Traitements médicamenteux  pré-opératoires :'
        if med_col in temp_df.columns:
            temp_df = augment_medications(temp_df, med_col)
        
        augmented_dfs.append(temp_df)
    
    final_df = pd.concat([df] + augmented_dfs, ignore_index=True)
    
    # Post-processing constraints
    if 'IMC (Kg/m2)' in final_df.columns:
        final_df['IMC (Kg/m2)'] = final_df['IMC (Kg/m2)'].clip(lower=15, upper=50)
    if 'Age lors de l\'intervention' in final_df.columns:
        final_df['Age lors de l\'intervention'] = final_df['Age lors de l\'intervention'].clip(lower=18, upper=100)
    
    return final_df

# Apply augmentation
final_augmented_df = medical_augmentation_pipeline(df, augmentation_factor=3)

TypeError: '>=' not supported between instances of 'datetime.datetime' and 'int'

In [33]:
print(aug_df.columns.tolist())

['Horodateur', 'nom prénom', 'numero du dossier', "date de l'intervention", "Age lors de l'intervention", 'sexe', 'IMC (Kg/m2)', 'SC (m2)', 'OBESITE', 'Comorbidités', 'Traitements médicamenteux  pré-opératoires :', 'Surface valvulaire aortique : cm²', 'Vitesse max :     m/s', 'Gradient moyen transvalvulaire : mmHg', 'Fraction d’éjection VG : %', 'Volume télédiastolique VG :  ml/m2', 'Volume télésystolique VG : ml/m2', 'Volume OG : ml/m2', 'PAPS : mmHg', 'Autres anomalies associées : ', 'Type de chirurgie :', 'Type de prothèse implantée', 'Marque et Numéro de prothèse ', 'Temps de circulation extracorporelle : min', 'Temps de clampage aortique : min', 'Vitesse max prothese : m/s', 'Gradient moyen protheses : mmHg', 'Surface de la prothèse : cm²', 'Surface indexée : cm²/m²', 'Fraction d’éjection VG post op: %', 'PAPS post-opératoire : mmHg', 'Complications post-opératoires précoces (< 30 jours)', 'Durée de séjour  en reanimation', 'Durée de séjour en unite post opératoire', 'Durée de séj

In [34]:
# Check data types
print(df.dtypes)

# Check mixed-type columns
for col in df.columns:
    unique_types = set(type(x) for x in df[col])
    if len(unique_types) > 1:
        print(f"Column '{col}' has mixed types: {unique_types}")

Horodateur                                              datetime64[ns]
nom prénom                                                      object
numero du dossier                                               object
date de l'intervention                                  datetime64[ns]
Age lors de l'intervention                                       int64
sexe                                                             int64
IMC (Kg/m2)                                                     object
SC (m2)                                                        float64
OBESITE                                                        float64
Comorbidités                                                     int64
Traitements médicamenteux  pré-opératoires :                    object
Surface valvulaire aortique : cm²                              float64
Vitesse max :     m/s                                          float64
Gradient moyen transvalvulaire : mmHg                            int64
Fracti